In [1]:
#%load_ext nb_black
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
spark=SparkSession.builder.getOrCreate()

In [3]:
# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
rdd = spark.read.csv("crime_data_filtered.csv",inferSchema=True, header=True)

In [8]:
rdd.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- CMPLNT_NUM: integer (nullable = true)
 |-- ADDR_PCT_CD: double (nullable = true)
 |-- BORO_NM: string (nullable = true)
 |-- CMPLNT_FR_DT: string (nullable = true)
 |-- RPT_DT: string (nullable = true)
 |-- CMPLNT_FR_TM: string (nullable = true)
 |-- CRM_ATPT_CPTD_CD: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- SUSP_AGE_GROUP: string (nullable = true)
 |-- SUSP_RACE: string (nullable = true)
 |-- SUSP_SEX: string (nullable = true)
 |-- VIC_AGE_GROUP: string (nullable = true)
 |-- VIC_RACE: string (nullable = true)
 |-- VIC_SEX: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [9]:
rdd.take(2)

[Row(_c0=0, CMPLNT_NUM=394506329, ADDR_PCT_CD=32.0, BORO_NM='MANHATTAN', CMPLNT_FR_DT='12/31/2019', RPT_DT='2019-12-31', CMPLNT_FR_TM='17:30:00', CRM_ATPT_CPTD_CD='COMPLETED', KY_CD=118, LAW_CAT_CD='FELONY', SUSP_AGE_GROUP=None, SUSP_RACE=None, SUSP_SEX=None, VIC_AGE_GROUP='UNKNOWN', VIC_RACE='UNKNOWN', VIC_SEX='E', Latitude=40.82092679700002, Longitude=-73.94332421899996),
 Row(_c0=1, CMPLNT_NUM=968873685, ADDR_PCT_CD=47.0, BORO_NM='BRONX', CMPLNT_FR_DT='12/29/2019', RPT_DT='2019-12-29', CMPLNT_FR_TM='16:31:00', CRM_ATPT_CPTD_CD='COMPLETED', KY_CD=113, LAW_CAT_CD='FELONY', SUSP_AGE_GROUP=None, SUSP_RACE=None, SUSP_SEX=None, VIC_AGE_GROUP='UNKNOWN', VIC_RACE='UNKNOWN', VIC_SEX='E', Latitude=40.88570140600008, Longitude=-73.86164032499995)]

In [10]:
rdd.show(1)

+---+----------+-----------+---------+------------+----------+------------+----------------+-----+----------+--------------+---------+--------+-------------+--------+-------+-----------------+------------------+
|_c0|CMPLNT_NUM|ADDR_PCT_CD|  BORO_NM|CMPLNT_FR_DT|    RPT_DT|CMPLNT_FR_TM|CRM_ATPT_CPTD_CD|KY_CD|LAW_CAT_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|         Latitude|         Longitude|
+---+----------+-----------+---------+------------+----------+------------+----------------+-----+----------+--------------+---------+--------+-------------+--------+-------+-----------------+------------------+
|  0| 394506329|       32.0|MANHATTAN|  12/31/2019|2019-12-31|    17:30:00|       COMPLETED|  118|    FELONY|          null|     null|    null|      UNKNOWN| UNKNOWN|      E|40.82092679700002|-73.94332421899996|
+---+----------+-----------+---------+------------+----------+------------+----------------+-----+----------+--------------+---------+--------+---------

In [11]:
rdd.select("vic_age_group","vic_sex").show(5)

+-------------+-------+
|vic_age_group|vic_sex|
+-------------+-------+
|      UNKNOWN|      E|
|      UNKNOWN|      E|
|        25-44|      F|
|      UNKNOWN|      E|
|      UNKNOWN|      M|
+-------------+-------+
only showing top 5 rows



In [27]:
df=rdd

In [28]:
df.columns

['_c0',
 'CMPLNT_NUM',
 'ADDR_PCT_CD',
 'BORO_NM',
 'CMPLNT_FR_DT',
 'RPT_DT',
 'CMPLNT_FR_TM',
 'CRM_ATPT_CPTD_CD',
 'KY_CD',
 'LAW_CAT_CD',
 'SUSP_AGE_GROUP',
 'SUSP_RACE',
 'SUSP_SEX',
 'VIC_AGE_GROUP',
 'VIC_RACE',
 'VIC_SEX',
 'Latitude',
 'Longitude']

In [29]:
df.show()

+---+----------+-----------+---------+------------+----------+------------+----------------+-----+-----------+--------------+--------------+--------+-------------+--------------+-------+------------------+------------------+
|_c0|CMPLNT_NUM|ADDR_PCT_CD|  BORO_NM|CMPLNT_FR_DT|    RPT_DT|CMPLNT_FR_TM|CRM_ATPT_CPTD_CD|KY_CD| LAW_CAT_CD|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|          Latitude|         Longitude|
+---+----------+-----------+---------+------------+----------+------------+----------------+-----+-----------+--------------+--------------+--------+-------------+--------------+-------+------------------+------------------+
|  0| 394506329|       32.0|MANHATTAN|  12/31/2019|2019-12-31|    17:30:00|       COMPLETED|  118|     FELONY|          null|          null|    null|      UNKNOWN|       UNKNOWN|      E| 40.82092679700002|-73.94332421899996|
|  1| 968873685|       47.0|    BRONX|  12/29/2019|2019-12-29|    16:31:00|       COMPLETED|  113|  

In [30]:
df.dtypes

[('_c0', 'int'),
 ('CMPLNT_NUM', 'int'),
 ('ADDR_PCT_CD', 'double'),
 ('BORO_NM', 'string'),
 ('CMPLNT_FR_DT', 'string'),
 ('RPT_DT', 'string'),
 ('CMPLNT_FR_TM', 'string'),
 ('CRM_ATPT_CPTD_CD', 'string'),
 ('KY_CD', 'int'),
 ('LAW_CAT_CD', 'string'),
 ('SUSP_AGE_GROUP', 'string'),
 ('SUSP_RACE', 'string'),
 ('SUSP_SEX', 'string'),
 ('VIC_AGE_GROUP', 'string'),
 ('VIC_RACE', 'string'),
 ('VIC_SEX', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double')]

In [31]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- CMPLNT_NUM: integer (nullable = true)
 |-- ADDR_PCT_CD: double (nullable = true)
 |-- BORO_NM: string (nullable = true)
 |-- CMPLNT_FR_DT: string (nullable = true)
 |-- RPT_DT: string (nullable = true)
 |-- CMPLNT_FR_TM: string (nullable = true)
 |-- CRM_ATPT_CPTD_CD: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- SUSP_AGE_GROUP: string (nullable = true)
 |-- SUSP_RACE: string (nullable = true)
 |-- SUSP_SEX: string (nullable = true)
 |-- VIC_AGE_GROUP: string (nullable = true)
 |-- VIC_RACE: string (nullable = true)
 |-- VIC_SEX: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [32]:
df.select('SUSP_AGE_GROUP','SUSP_SEX').show(10)

+--------------+--------+
|SUSP_AGE_GROUP|SUSP_SEX|
+--------------+--------+
|          null|    null|
|          null|    null|
|         25-44|       M|
|         18-24|       M|
|          null|    null|
|          null|    null|
|          null|    null|
|         18-24|       F|
|          null|    null|
|          null|    null|
+--------------+--------+
only showing top 10 rows



In [37]:
df.groupBy("CRM_ATPT_CPTD_CD").count().show()

+----------------+-------+
|CRM_ATPT_CPTD_CD|  count|
+----------------+-------+
|       ATTEMPTED|  38157|
|       COMPLETED|2247575|
+----------------+-------+



In [38]:
df.describe()

DataFrame[summary: string, _c0: string, CMPLNT_NUM: string, ADDR_PCT_CD: string, BORO_NM: string, CMPLNT_FR_DT: string, RPT_DT: string, CMPLNT_FR_TM: string, CRM_ATPT_CPTD_CD: string, KY_CD: string, LAW_CAT_CD: string, SUSP_AGE_GROUP: string, SUSP_RACE: string, SUSP_SEX: string, VIC_AGE_GROUP: string, VIC_RACE: string, VIC_SEX: string, Latitude: string, Longitude: string]

In [39]:
df.describe().show()

+-------+------------------+-------------------+------------------+-------------+------------+----------+------------+----------------+------------------+----------+------------------+--------------------+--------+------------------+--------------------+-------+-------------------+-------------------+
|summary|               _c0|         CMPLNT_NUM|       ADDR_PCT_CD|      BORO_NM|CMPLNT_FR_DT|    RPT_DT|CMPLNT_FR_TM|CRM_ATPT_CPTD_CD|             KY_CD|LAW_CAT_CD|    SUSP_AGE_GROUP|           SUSP_RACE|SUSP_SEX|     VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|           Latitude|          Longitude|
+-------+------------------+-------------------+------------------+-------------+------------+----------+------------+----------------+------------------+----------+------------------+--------------------+--------+------------------+--------------------+-------+-------------------+-------------------+
|  count|           2285732|            2285732|           2285729|      2283987|     22857

In [41]:
df.take(2)

[Row(_c0=0, CMPLNT_NUM=394506329, ADDR_PCT_CD=32.0, BORO_NM='MANHATTAN', CMPLNT_FR_DT='12/31/2019', RPT_DT='2019-12-31', CMPLNT_FR_TM='17:30:00', CRM_ATPT_CPTD_CD='COMPLETED', KY_CD=118, LAW_CAT_CD='FELONY', SUSP_AGE_GROUP=None, SUSP_RACE=None, SUSP_SEX=None, VIC_AGE_GROUP='UNKNOWN', VIC_RACE='UNKNOWN', VIC_SEX='E', Latitude=40.82092679700002, Longitude=-73.94332421899996),
 Row(_c0=1, CMPLNT_NUM=968873685, ADDR_PCT_CD=47.0, BORO_NM='BRONX', CMPLNT_FR_DT='12/29/2019', RPT_DT='2019-12-29', CMPLNT_FR_TM='16:31:00', CRM_ATPT_CPTD_CD='COMPLETED', KY_CD=113, LAW_CAT_CD='FELONY', SUSP_AGE_GROUP=None, SUSP_RACE=None, SUSP_SEX=None, VIC_AGE_GROUP='UNKNOWN', VIC_RACE='UNKNOWN', VIC_SEX='E', Latitude=40.88570140600008, Longitude=-73.86164032499995)]

In [43]:
# Re-order and select columns
df = df.select('CMPLNT_NUM',
 'ADDR_PCT_CD',
 'BORO_NM',
 'CMPLNT_FR_DT',
 'RPT_DT',
 'CMPLNT_FR_TM',
 'CRM_ATPT_CPTD_CD',
 'KY_CD',
 'LAW_CAT_CD',
 'SUSP_AGE_GROUP',
 'SUSP_RACE',
 'SUSP_SEX',
 'VIC_AGE_GROUP',
 'VIC_RACE',
 'VIC_SEX',
 'Latitude',
 'Longitude')

In [44]:
df.columns

['CMPLNT_NUM',
 'ADDR_PCT_CD',
 'BORO_NM',
 'CMPLNT_FR_DT',
 'RPT_DT',
 'CMPLNT_FR_TM',
 'CRM_ATPT_CPTD_CD',
 'KY_CD',
 'LAW_CAT_CD',
 'SUSP_AGE_GROUP',
 'SUSP_RACE',
 'SUSP_SEX',
 'VIC_AGE_GROUP',
 'VIC_RACE',
 'VIC_SEX',
 'Latitude',
 'Longitude']

In [80]:
df1.show(10)

+----------+----------------+
|CMPLNT_NUM|CRM_ATPT_CPTD_CD|
+----------+----------------+
| 394506329|       COMPLETED|
| 968873685|       COMPLETED|
| 509837549|       COMPLETED|
| 352454313|       COMPLETED|
| 293718737|       ATTEMPTED|
| 552685226|       COMPLETED|
| 134037758|       COMPLETED|
| 855385879|       COMPLETED|
| 241602326|       COMPLETED|
| 585574881|       COMPLETED|
+----------+----------------+
only showing top 10 rows



In [82]:
import pandas as pd
df=pd.read_csv("crime_data_filtered.csv")

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2285732 entries, 0 to 2285731
Data columns (total 18 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Unnamed: 0        int64  
 1   CMPLNT_NUM        int64  
 2   ADDR_PCT_CD       float64
 3   BORO_NM           object 
 4   CMPLNT_FR_DT      object 
 5   RPT_DT            object 
 6   CMPLNT_FR_TM      object 
 7   CRM_ATPT_CPTD_CD  object 
 8   KY_CD             int64  
 9   LAW_CAT_CD        object 
 10  SUSP_AGE_GROUP    object 
 11  SUSP_RACE         object 
 12  SUSP_SEX          object 
 13  VIC_AGE_GROUP     object 
 14  VIC_RACE          object 
 15  VIC_SEX           object 
 16  Latitude          float64
 17  Longitude         float64
dtypes: float64(3), int64(3), object(12)
memory usage: 313.9+ MB


In [96]:

df.loc[df['CRM_ATPT_CPTD_CD'] == 'COMPLETED'] = 1
df.loc[df['CRM_ATPT_CPTD_CD'] == 'ATTEMPTED'] = 0

In [97]:
df=df.astype({'CRM_ATPT_CPTD_CD':int})

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2285732 entries, 0 to 2285731
Data columns (total 18 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Unnamed: 0        int64  
 1   CMPLNT_NUM        int64  
 2   ADDR_PCT_CD       float64
 3   BORO_NM           object 
 4   CMPLNT_FR_DT      object 
 5   RPT_DT            object 
 6   CMPLNT_FR_TM      object 
 7   CRM_ATPT_CPTD_CD  int32  
 8   KY_CD             int64  
 9   LAW_CAT_CD        object 
 10  SUSP_AGE_GROUP    object 
 11  SUSP_RACE         object 
 12  SUSP_SEX          object 
 13  VIC_AGE_GROUP     object 
 14  VIC_RACE          object 
 15  VIC_SEX           object 
 16  Latitude          float64
 17  Longitude         float64
dtypes: float64(3), int32(1), int64(3), object(11)
memory usage: 305.2+ MB


In [103]:
X=df.drop(['CRM_ATPT_CPTD_CD','BORO_NM','CMPLNT_FR_DT','RPT_DT','CMPLNT_FR_TM','LAW_CAT_CD','SUSP_AGE_GROUP','SUSP_RACE', 'SUSP_SEX','VIC_AGE_GROUP','VIC_RACE','VIC_SEX'],axis=1)

In [104]:
y=df['CRM_ATPT_CPTD_CD']

In [105]:
from sklearn.model_selection import train_test_split

In [106]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.20, random_state=30)

In [107]:
from sklearn.linear_model import LinearRegression

In [108]:
model=LinearRegression()    #initialising my model

In [109]:
model.fit(X_train,y_train)

LinearRegression()

In [110]:
pred = model.predict(X_test)

In [111]:
pred

array([1., 1., 1., ..., 1., 1., 1.])

In [112]:
model.score(X_train,y_train)

1.0

In [113]:
model.score(X_test,y_test)

1.0

In [114]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,pred)

3.192703484736402e-27